# 病床姿勢監測 - 資料準備與對齊

**專案**: 熱成像超解析度 + 姿勢辨識  
**作者**: rochi190  
**日期**: 2025-11-03  

## 功能
1. 讀取 output.txt 取得相機開始時間
2. 載入熱成像 log 檔案（多核加速）
3. 分析 RGB 影片資訊
4. **智能配對**（你的方法）：
   - 不管 FPS 波動，只要時間匹配就配對
   - Burst 資料（一秒50+張）分配給前面缺資料的區段
   - 只在時間容忍度內配對
5. 翻轉檢測與修復
6. 匯出 RGB 圖片供 LabelMe 標註（多核加速）
7. 儲存對齊後的 Thermal 資料

## 輸出
- `output/labelme_project/` - LabelMe 標註專案
- `output/aligned_dataset/` - 對齊後的訓練資料

---
## Step 0: 環境設定與套件匯入

In [1]:
import os
import sys
import json
import re
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime, timedelta
from typing import List, Dict, Tuple, Optional, Any
from tqdm import tqdm
from dataclasses import dataclass, field
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from collections import defaultdict, Counter
import multiprocessing as mp
import pickle
import warnings
warnings.filterwarnings('ignore')

# Matplotlib 設定
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.dpi'] = 100

# Seaborn 樣式
sns.set_style('whitegrid')

print("="*70)
print("病床姿勢監測系統 - 資料準備模組")
print("="*70)
print(f"執行時間: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"使用者: rochi190")
print(f"Python 版本: {sys.version}")
print(f"NumPy 版本: {np.__version__}")
print(f"OpenCV 版本: {cv2.__version__}")
print("="*70)

病床姿勢監測系統 - 資料準備模組
執行時間: 2025-11-04 04:11:19
使用者: rochi190
Python 版本: 3.11.13 (main, Jun  5 2025, 13:12:00) [GCC 11.2.0]
NumPy 版本: 1.26.4
OpenCV 版本: 4.10.0


---
## Step 1: 路徑與參數配置

In [ ]:
# ========================================
# 多核處理配置
# ========================================
NUM_WORKERS = max(1, mp.cpu_count() - 1)  # 保留一個核心給系統
CHUNK_SIZE = 100  # 批次處理大小

print(f"🚀 多核處理配置:")
print(f"  系統 CPU 核心數: {mp.cpu_count()}")
print(f"  使用 Workers: {NUM_WORKERS}")
print(f"  批次大小: {CHUNK_SIZE}")

# ========================================
# 路徑配置
# ========================================
BASE_DIR = Path(os.getcwd())
DATA_DIR = BASE_DIR / 'data'
THERMAL_DIR = DATA_DIR / 'thermal'
CAMERA_DIR = DATA_DIR / 'camera'
OUTPUT_TXT = CAMERA_DIR / 'output.txt'
AVI_FILE = CAMERA_DIR / 'output.avi'

# 輸出目錄
OUTPUT_DIR = BASE_DIR / 'output'
LABELME_DIR = OUTPUT_DIR / 'labelme_project'
ALIGNED_DIR = OUTPUT_DIR / 'aligned_dataset'
DIAGNOSTIC_DIR = OUTPUT_DIR / 'diagnostics'

# 建立所有輸出目錄
for directory in [LABELME_DIR, ALIGNED_DIR, DIAGNOSTIC_DIR]:
    directory.mkdir(parents=True, exist_ok=True)
    print(f"  ✓ {directory.relative_to(BASE_DIR)}")

# ========================================
# 熱成像參數
# ========================================
THERMAL_RESOLUTION = (32, 24)  # (寬度, 高度)
THERMAL_WIDTH, THERMAL_HEIGHT = THERMAL_RESOLUTION

# ========================================
# 智能配對參數（你的方法）
# ========================================
TARGET_FPS = 8  # 目標 FPS（僅供參考）
FRAME_INTERVAL = 1.0 / TARGET_FPS  # 0.125 秒

# 時間容忍度：半個幀間隔
TIMESTAMP_TOLERANCE = FRAME_INTERVAL / 2  # 62.5ms

# Burst 檢測閾值：一秒內超過此數量視為 burst
BURST_THRESHOLD = 15

# 最小幀間隔：避免重複幀（20ms）
MIN_FRAME_INTERVAL = 0.02

# ========================================
# 翻轉設定（執行 Step 6 後調整）
# ========================================
FLIP_THERMAL = False  # 是否翻轉熱成像
FLIP_RGB = False      # 是否翻轉 RGB

# ========================================
# 顯示配置
# ========================================
print(f"\n📁 路徑檢查:")
print(f"  Thermal 目錄: {'✓' if THERMAL_DIR.exists() else '✗'} {THERMAL_DIR}")
print(f"  Camera 目錄: {'✓' if CAMERA_DIR.exists() else '✗'} {CAMERA_DIR}")
print(f"  output.txt: {'✓' if OUTPUT_TXT.exists() else '✗'} {OUTPUT_TXT}")
print(f"  AVI 檔案: {'✓' if AVI_FILE.exists() else '✗'} {AVI_FILE}")

print(f"\n⚙️ 配對參數:")
print(f"  目標 FPS: {TARGET_FPS}")
print(f"  幀間隔: {FRAME_INTERVAL*1000:.1f}ms")
print(f"  時間容忍度: {TIMESTAMP_TOLERANCE*1000:.1f}ms")
print(f"  Burst 閾值: >{BURST_THRESHOLD} 張/秒")
print(f"  最小幀間隔: {MIN_FRAME_INTERVAL*1000:.1f}ms")

print(f"\n🔄 翻轉設定:")
print(f"  翻轉 Thermal: {FLIP_THERMAL}")
print(f"  翻轉 RGB: {FLIP_RGB}")

---
## Step 2: 資料結構定義

In [ ]:
@dataclass
class TimeReference:
    """時間參考資料結構"""
    camera_start: datetime
    events: List[Dict] = field(default_factory=list)
    
    def __repr__(self):
        return f"TimeReference(camera_start={self.camera_start}, events={len(self.events)})"

@dataclass
class VideoInfo:
    """RGB 影片資訊"""
    fps: float
    frame_count: int
    duration: float
    width: int
    height: int
    start_time: datetime
    end_time: datetime
    
    def __repr__(self):
        return (f"VideoInfo(fps={self.fps:.2f}, frames={self.frame_count}, "
                f"duration={self.duration:.2f}s, resolution={self.width}x{self.height})")

@dataclass
class ThermalFrame:
    """熱成像幀資料"""
    timestamp: datetime
    image: np.ndarray
    file: str
    line: int = -1
    
    def __repr__(self):
        return f"ThermalFrame(timestamp={self.timestamp}, shape={self.image.shape}, file={self.file})"

@dataclass
class AlignedPair:
    """對齊後的配對資料"""
    pair_id: str
    timestamp: datetime
    thermal_image: np.ndarray
    thermal_file: str
    rgb_frame_idx: int
    rgb_error_ms: float
    thermal_error_ms: float = 0.0
    
    def to_dict(self) -> Dict:
        return {
            'pair_id': self.pair_id,
            'timestamp': self.timestamp,
            'thermal_file': self.thermal_file,
            'rgb_frame_idx': self.rgb_frame_idx,
            'rgb_error_ms': self.rgb_error_ms,
            'thermal_error_ms': self.thermal_error_ms
        }

print("✅ 資料結構定義完成")

---
## Step 3: 解析 output.txt（相機開始時間）

In [ ]:
def parse_output_txt(txt_path: Path) -> Optional[TimeReference]:
    """
    解析 output.txt 取得相機開始時間
    
    支援格式:
    - YYYY-MM-DD HH:MM:SS
    - YYYY/MM/DD HH:MM:SS
    - Unix timestamp (10位數字)
    
    Args:
        txt_path: output.txt 路徑
        
    Returns:
        TimeReference 物件或 None
    """
    if not txt_path.exists():
        print(f"❌ {txt_path} 不存在")
        return None
    
    # 嘗試多種編碼
    content = None
    for encoding in ['utf-8', 'big5', 'gbk', 'latin1']:
        try:
            with open(txt_path, 'r', encoding=encoding) as f:
                content = f.read()
            break
        except UnicodeDecodeError:
            continue
    
    if content is None:
        print(f"❌ 無法讀取檔案（編碼問題）")
        return None
    
    print(f"\n{'='*70}")
    print(f"解析 output.txt")
    print(f"{'='*70}")
    print(f"檔案大小: {len(content)} bytes")
    print(f"總行數: {len(content.splitlines())}")
    print(f"\n檔案內容（前 500 字元）:")
    print("-" * 70)
    print(content[:500])
    print("-" * 70)
    
    camera_start = None
    events = []
    
    # 時間格式模式
    patterns = [
        (r'(\d{4}-\d{2}-\d{2}\s+\d{2}:\d{2}:\d{2})', '%Y-%m-%d %H:%M:%S', 'ISO format'),
        (r'(\d{4}/\d{2}/\d{2}\s+\d{2}:\d{2}:\d{2})', '%Y/%m/%d %H:%M:%S', 'Slash format'),
        (r'(\d{10})', 'unix', 'Unix timestamp')
    ]
    
    for line_num, line in enumerate(content.strip().split('\n'), 1):
        line = line.strip()
        if not line:
            continue
        
        for pattern, fmt, desc in patterns:
            match = re.search(pattern, line)
            if match:
                time_str = match.group(1)
                
                try:
                    if fmt == 'unix':
                        timestamp = datetime.fromtimestamp(int(time_str))
                    else:
                        timestamp = datetime.strptime(time_str, fmt)
                    
                    # 判斷是否為開始時間
                    keywords = ['start', 'video', 'begin', '開始', 'camera']
                    if any(keyword in line.lower() for keyword in keywords):
                        camera_start = timestamp
                        print(f"✅ 找到相機開始時間（第 {line_num} 行）: {camera_start}")
                        print(f"   格式: {desc}")
                        print(f"   原始行: {line}")
                    
                    events.append({
                        'timestamp': timestamp,
                        'description': line,
                        'line_number': line_num,
                        'format': desc
                    })
                    break
                    
                except Exception as e:
                    print(f"⚠️ 解析失敗（第 {line_num} 行）: {e}")
                    continue
    
    # Fallback: 使用第一個事件時間
    if camera_start is None and events:
        camera_start = events[0]['timestamp']
        print(f"\n⚠️ 未找到明確開始時間，使用第一個事件: {camera_start}")
        print(f"   來源: {events[0]['description'][:50]}...")
    
    if camera_start is None:
        print(f"\n❌ 無法解析時間，請檢查檔案格式")
        return None
    
    # 排序事件
    events.sort(key=lambda x: x['timestamp'])
    
    print(f"\n✅ 解析成功")
    print(f"  相機開始時間: {camera_start}")
    print(f"  解析到 {len(events)} 個事件")
    if events:
        print(f"  事件時間範圍: {events[0]['timestamp']} ~ {events[-1]['timestamp']}")
    print("="*70)
    
    return TimeReference(camera_start=camera_start, events=events)

# ========================================
# 執行解析
# ========================================
time_ref = parse_output_txt(OUTPUT_TXT)

if time_ref is None:
    raise RuntimeError("❌ 無法解析 output.txt，請檢查檔案格式或路徑")

print(f"\n📌 時間參考: {time_ref}")

---
## Step 4: 載入熱成像資料（多核加速）

In [ ]:
def parse_single_thermal_file(file_path: Path) -> List[ThermalFrame]:
    """
    解析單一熱成像 log 檔案（用於多核處理）
    
    Args:
        file_path: log 檔案路徑
        
    Returns:
        ThermalFrame 物件列表
    """
    frames = []
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        
        for line_idx, line in enumerate(lines):
            line = line.strip()
            if not line:
                continue
            
            try:
                # 解析 JSON
                data_dict = json.loads(line)
                
                # 解析時間戳 [year, month, day, hour, minute, second, weekday, millisecond]
                time_parts = data_dict.get('time', [])
                if len(time_parts) < 6:
                    continue
                
                year, month, day, hour, minute, second = time_parts[:6]
                
                # 處理毫秒（可能在索引 6 或 7）
                millisecond = 0
                if len(time_parts) >= 7:
                    # 檢查是 weekday (0-6) 還是 millisecond
                    if time_parts[6] < 7:
                        # 是 weekday，毫秒在索引 7
                        millisecond = time_parts[7] if len(time_parts) > 7 else 0
                    else:
                        # 是毫秒
                        millisecond = time_parts[6]
                
                # 建立時間戳
                try:
                    timestamp = datetime(
                        year, month, day,
                        hour, minute, second,
                        millisecond * 1000  # 轉為 microseconds
                    )
                except ValueError:
                    # 日期時間無效
                    continue
                
                # 解析熱成像圖像
                message = data_dict.get('message', [])
                expected_size = THERMAL_WIDTH * THERMAL_HEIGHT
                
                if len(message) != expected_size:
                    continue
                
                thermal_image = np.array(message, dtype=np.uint8).reshape(
                    THERMAL_HEIGHT, THERMAL_WIDTH
                )
                
                frames.append(ThermalFrame(
                    timestamp=timestamp,
                    image=thermal_image,
                    file=file_path.name,
                    line=line_idx
                ))
                
            except (json.JSONDecodeError, KeyError, ValueError, IndexError):
                # 解析失敗，跳過這一行
                continue
    
    except Exception as e:
        print(f"❌ 載入檔案失敗 {file_path.name}: {e}")
        return []
    
    return frames

def load_thermal_data_multicore(thermal_dir: Path, num_workers: int = NUM_WORKERS) -> List[ThermalFrame]:
    """
    多核並行載入所有熱成像資料
    
    Args:
        thermal_dir: 熱成像目錄
        num_workers: Worker 數量
        
    Returns:
        ThermalFrame 物件列表（已排序）
    """
    log_files = sorted(thermal_dir.glob('log*.txt'))
    
    if not log_files:
        raise FileNotFoundError(f"❌ 未找到 log 檔案於 {thermal_dir}")
    
    print(f"\n{'='*70}")
    print(f"載入熱成像資料（多核: {num_workers} workers）")
    print(f"{'='*70}")
    print(f"找到 {len(log_files)} 個 log 檔案")
    
    # 顯示前幾個檔案
    print("\n檔案列表:")
    for i, f in enumerate(log_files[:5], 1):
        print(f"  {i}. {f.name}")
    if len(log_files) > 5:
        print(f"  ... 還有 {len(log_files)-5} 個檔案")
    
    # 多核處理
    print(f"\n開始並行解析...")
    all_frames = []
    
    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        results = list(tqdm(
            executor.map(parse_single_thermal_file, log_files),
            total=len(log_files),
            desc="解析 Thermal 檔案",
            unit="file"
        ))
    
    # 合併結果
    for result in results:
        all_frames.extend(result)
    
    if not all_frames:
        raise ValueError("❌ 未載入任何熱成像資料")
    
    # 依時間戳排序
    all_frames.sort(key=lambda x: x.timestamp)
    
    # 統計資訊
    thermal_start = all_frames[0].timestamp
    thermal_end = all_frames[-1].timestamp
    duration = (thermal_end - thermal_start).total_seconds()
    avg_fps = len(all_frames) / duration if duration > 0 else 0
    
    print(f"\n✅ 載入完成")
    print(f"  總幀數: {len(all_frames):,}")
    print(f"  時間範圍: {thermal_start} ~ {thermal_end}")
    print(f"  時長: {duration:.2f} 秒 ({duration/60:.2f} 分鐘)")
    print(f"  平均 FPS: {avg_fps:.2f}")
    print("="*70)
    
    return all_frames

# ========================================
# 執行載入
# ========================================
thermal_frames = load_thermal_data_multicore(THERMAL_DIR)

print(f"\n📌 熱成像資料: {len(thermal_frames):,} 幀")

---
## Step 5: 分析 RGB 影片

In [ ]:
def analyze_video(avi_path: Path, camera_start: datetime) -> VideoInfo:
    """
    分析 RGB 影片並計算時間範圍
    
    Args:
        avi_path: AVI 檔案路徑
        camera_start: 相機開始時間
        
    Returns:
        VideoInfo 物件
    """
    if not avi_path.exists():
        raise FileNotFoundError(f"❌ {avi_path} 不存在")
    
    cap = cv2.VideoCapture(str(avi_path))
    
    if not cap.isOpened():
        raise RuntimeError(f"❌ 無法開啟影片 {avi_path}")
    
    # 取得影片資訊
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    duration = frame_count / fps if fps > 0 else 0
    
    start_time = camera_start
    end_time = start_time + timedelta(seconds=duration)
    
    # 測試讀取第一幀
    ret, first_frame = cap.read()
    first_frame_ok = ret
    
    cap.release()
    
    print(f"\n{'='*70}")
    print(f"RGB 影片資訊")
    print(f"{'='*70}")
    print(f"  檔案: {avi_path.name}")
    print(f"  FPS: {fps:.2f}")
    print(f"  總幀數: {frame_count:,}")
    print(f"  解析度: {width} × {height}")
    print(f"  時長: {duration:.2f} 秒 ({duration/60:.2f} 分鐘)")
    print(f"  第一幀可讀取: {'✓' if first_frame_ok else '✗'}")
    print(f"\n  時間範圍:")
    print(f"    開始: {start_time}")
    print(f"    結束: {end_time}")
    print("="*70)
    
    return VideoInfo(
        fps=fps,
        frame_count=frame_count,
        duration=duration,
        width=width,
        height=height,
        start_time=start_time,
        end_time=end_time
    )

# ========================================
# 執行分析
# ========================================
video_info = analyze_video(AVI_FILE, time_ref.camera_start)

print(f"\n📌 影片資訊: {video_info}")

---
## Step 6: 智能配對演算法（你的方法）

In [ ]:
def analyze_thermal_distribution(thermal_frames: List[ThermalFrame]) -> Dict[str, Any]:
    """
    分析熱成像資料的時間分佈
    找出 burst periods 和 time gaps
    
    Args:
        thermal_frames: 熱成像幀列表
        
    Returns:
        包含統計資訊的字典
    """
    print(f"\n{'='*70}")
    print(f"分析熱成像時間分佈")
    print(f"{'='*70}")
    
    # 計算每秒幀數
    frames_per_second = defaultdict(list)
    for frame in thermal_frames:
        second_key = frame.timestamp.replace(microsecond=0)
        frames_per_second[second_key].append(frame)
    
    fps_counts = {k: len(v) for k, v in frames_per_second.items()}
    
    # 找出 burst periods（一秒內超過閾值）
    burst_periods = {k: v for k, v in fps_counts.items() if v > BURST_THRESHOLD}
    
    # 找出 gaps（相鄰秒數間隔 > 1.5 秒）
    sorted_seconds = sorted(frames_per_second.keys())
    gaps = []
    for i in range(len(sorted_seconds) - 1):
        diff = (sorted_seconds[i+1] - sorted_seconds[i]).total_seconds()
        if diff > 1.5:
            gaps.append({
                'start': sorted_seconds[i],
                'end': sorted_seconds[i+1],
                'duration': diff
            })
    
    # 統計
    fps_values = list(fps_counts.values())
    
    print(f"\n📊 時間分佈統計:")
    print(f"  總秒數: {len(frames_per_second)}")
    print(f"  平均 FPS: {np.mean(fps_values):.2f}")
    print(f"  中位數 FPS: {np.median(fps_values):.2f}")
    print(f"  FPS 範圍: {min(fps_values)} ~ {max(fps_values)}")
    print(f"  標準差: {np.std(fps_values):.2f}")
    
    print(f"\n⚡ Burst Periods（>{BURST_THRESHOLD} 張/秒）:")
    print(f"  檢測到 {len(burst_periods)} 個 burst periods")
    if burst_periods:
        sorted_bursts = sorted(burst_periods.items(), key=lambda x: x[1], reverse=True)
        for second, count in sorted_bursts[:5]:
            print(f"    {second}: {count} 張")
        if len(burst_periods) > 5:
            print(f"    ... 還有 {len(burst_periods)-5} 個")
    
    print(f"\n🕳️ Time Gaps（>1.5秒）:")
    print(f"  檢測到 {len(gaps)} 個 gaps")
    if gaps:
        for gap in gaps[:5]:
            print(f"    {gap['start'].strftime('%H:%M:%S')} ~ {gap['end'].strftime('%H:%M:%S')}: {gap['duration']:.1f}秒")
        if len(gaps) > 5:
            print(f"    ... 還有 {len(gaps)-5} 個")
    
    print("="*70)
    
    return {
        'frames_per_second': frames_per_second,
        'fps_counts': fps_counts,
        'burst_periods': burst_periods,
        'gaps': gaps,
        'stats': {
            'mean_fps': np.mean(fps_values),
            'median_fps': np.median(fps_values),
            'min_fps': min(fps_values),
            'max_fps': max(fps_values),
            'std_fps': np.std(fps_values)
        }
    }

def find_overlap_period(
    thermal_frames: List[ThermalFrame],
    video_info: VideoInfo
) -> Tuple[datetime, datetime]:
    """
    計算 Thermal 和 RGB 的重疊時間區間
    
    Args:
        thermal_frames: 熱成像幀列表
        video_info: RGB 影片資訊
        
    Returns:
        (overlap_start, overlap_end)
    """
    thermal_start = thermal_frames[0].timestamp
    thermal_end = thermal_frames[-1].timestamp
    
    rgb_start = video_info.start_time
    rgb_end = video_info.end_time
    
    overlap_start = max(thermal_start, rgb_start)
    overlap_end = min(thermal_end, rgb_end)
    
    print(f"\n{'='*70}")
    print(f"計算重疊時間區間")
    print(f"{'='*70}")
    
    print(f"\nThermal 時間範圍:")
    print(f"  開始: {thermal_start}")
    print(f"  結束: {thermal_end}")
    thermal_duration = (thermal_end - thermal_start).total_seconds()
    print(f"  時長: {thermal_duration:.2f} 秒 ({thermal_duration/60:.2f} 分鐘)")
    
    print(f"\nRGB 時間範圍:")
    print(f"  開始: {rgb_start}")
    print(f"  結束: {rgb_end}")
    print(f"  時長: {video_info.duration:.2f} 秒 ({video_info.duration/60:.2f} 分鐘)")
    
    if overlap_start >= overlap_end:
        print(f"\n❌ 錯誤: 無重疊區間！")
        print(f"  Thermal 結束於: {thermal_end}")
        print(f"  RGB 開始於: {rgb_start}")
        time_diff = (rgb_start - thermal_end).total_seconds()
        print(f"  時間差: {abs(time_diff):.2f} 秒")
        raise ValueError("Thermal 和 RGB 時間不重疊，無法配對")
    
    overlap_duration = (overlap_end - overlap_start).total_seconds()
    
    print(f"\n✅ 重疊區間:")
    print(f"  開始: {overlap_start}")
    print(f"  結束: {overlap_end}")
    print(f"  時長: {overlap_duration:.2f} 秒 ({overlap_duration/60:.2f} 分鐘)")
    print(f"  預期最大配對數（@{TARGET_FPS}fps）: {int(overlap_duration * TARGET_FPS):,}")
    print("="*70)
    
    return overlap_start, overlap_end

def intelligent_pairing(
    thermal_frames: List[ThermalFrame],
    video_info: VideoInfo,
    overlap_start: datetime,
    overlap_end: datetime,
    tolerance: float = TIMESTAMP_TOLERANCE
) -> List[AlignedPair]:
    """
    智能配對演算法（你的方法）
    
    策略：
    1. 只配對重疊區間內的資料
    2. 不管 FPS 多少，只要時間匹配就配對
    3. Burst 資料自動分散
    4. 每個 RGB 幀只配對一次
    
    Args:
        thermal_frames: 熱成像幀列表
        video_info: RGB 影片資訊
        overlap_start: 重疊區間開始時間
        overlap_end: 重疊區間結束時間
        tolerance: 時間容忍度（秒）
        
    Returns:
        AlignedPair 物件列表
    """
    print(f"\n{'='*70}")
    print(f"智能配對演算法")
    print(f"{'='*70}")
    print(f"  時間容忍度: {tolerance*1000:.1f}ms")
    print(f"  重疊區間: {overlap_start} ~ {overlap_end}")
    
    # 過濾重疊區間內的 thermal 資料
    thermal_in_overlap = [
        frame for frame in thermal_frames
        if overlap_start <= frame.timestamp <= overlap_end
    ]
    
    print(f"\n重疊區間內 Thermal 幀數: {len(thermal_in_overlap):,}")
    
    # RGB 幀索引計算函數
    def get_rgb_frame_index(target_time: datetime) -> Optional[Tuple[int, float]]:
        """計算最接近目標時間的 RGB 幀索引"""
        offset = (target_time - video_info.start_time).total_seconds()
        
        # 邊界檢查
        if offset < 0 or offset > video_info.duration:
            return None
        
        # 計算幀索引（四捨五入）
        exact_frame = offset * video_info.fps
        frame_idx = int(round(exact_frame))
        
        # 確保在有效範圍內
        frame_idx = max(0, min(frame_idx, video_info.frame_count - 1))
        
        # 計算實際時間誤差
        actual_time_offset = frame_idx / video_info.fps
        error = abs(actual_time_offset - offset)
        
        # 檢查是否在容忍度內
        if error <= tolerance:
            return frame_idx, error
        else:
            return None
    
    # 配對
    pairs = []
    used_rgb_frames = set()
    skipped_count = 0
    
    print(f"\n開始配對...")
    for thermal_frame in tqdm(thermal_in_overlap, desc="配對 Thermal & RGB", unit="frame"):
        result = get_rgb_frame_index(thermal_frame.timestamp)
        
        if result is None:
            skipped_count += 1
            continue
        
        rgb_idx, rgb_error = result
        
        # 避免重複使用同一個 RGB 幀
        if rgb_idx in used_rgb_frames:
            skipped_count += 1
            continue
        
        used_rgb_frames.add(rgb_idx)
        
        pairs.append(AlignedPair(
            pair_id=f"pair_{len(pairs):05d}",
            timestamp=thermal_frame.timestamp,
            thermal_image=thermal_frame.image,
            thermal_file=thermal_frame.file,
            rgb_frame_idx=rgb_idx,
            rgb_error_ms=rgb_error * 1000,
            thermal_error_ms=0.0  # Thermal 為基準
        ))
    
    print(f"\n✅ 配對完成")
    print(f"  成功配對: {len(pairs):,} 對")
    print(f"  跳過: {skipped_count:,} 幀")
    print(f"  利用率: {len(pairs) / len(thermal_in_overlap) * 100:.1f}%")
    
    if pairs:
        rgb_errors = [p.rgb_error_ms for p in pairs]
        print(f"\n  RGB 時間誤差統計:")
        print(f"    平均: {np.mean(rgb_errors):.2f}ms")
        print(f"    中位數: {np.median(rgb_errors):.2f}ms")
        print(f"    最大: {np.max(rgb_errors):.2f}ms")
        print(f"    最小: {np.min(rgb_errors):.2f}ms")
        print(f"    標準差: {np.std(rgb_errors):.2f}ms")
    
    print("="*70)
    
    return pairs

# ========================================
# 執行分析與配對
# ========================================
distribution_stats = analyze_thermal_distribution(thermal_frames)
overlap_start, overlap_end = find_overlap_period(thermal_frames, video_info)
aligned_pairs = intelligent_pairing(thermal_frames, video_info, overlap_start, overlap_end)

print(f"\n📌 對齊結果: {len(aligned_pairs):,} 對")

---
## Step 7: 翻轉檢測

In [ ]:
def check_flip_orientation(
    pairs: List[AlignedPair],
    avi_path: Path,
    num_samples: int = 6
) -> None:
    """
    檢查是否需要左右翻轉
    顯示 4 種組合供人工判斷
    
    Args:
        pairs: 對齊配對列表
        avi_path: AVI 檔案路徑
        num_samples: 取樣數量
    """
    if not pairs:
        print("❌ 無配對資料可檢查")
        return
    
    print(f"\n{'='*70}")
    print(f"翻轉方向檢測")
    print(f"{'='*70}")
    print(f"取樣數量: {num_samples}")
    
    # 均勻取樣
    indices = np.linspace(0, len(pairs)-1, num_samples, dtype=int)
    sample_pairs = [pairs[i] for i in indices]
    
    cap = cv2.VideoCapture(str(avi_path))
    
    # 4 種組合 × 2 (thermal+rgb) = 8 欄
    fig, axes = plt.subplots(num_samples, 8, figsize=(24, num_samples*3))
    
    print(f"\n讀取樣本...")
    for row_idx, pair in enumerate(tqdm(sample_pairs, desc="處理樣本")):
        thermal = pair.thermal_image
        
        # 讀取 RGB
        cap.set(cv2.CAP_PROP_POS_FRAMES, pair.rgb_frame_idx)
        ret, rgb = cap.read()
        
        if not ret:
            print(f"⚠️ 無法讀取 RGB 幀 {pair.rgb_frame_idx}")
            continue
        
        rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
        
        # 4 種組合
        combinations = [
            (thermal, rgb, "原始", "Both Original"),
            (cv2.flip(thermal, 1), rgb, "T翻轉", "Thermal Flipped"),
            (thermal, cv2.flip(rgb, 1), "R翻轉", "RGB Flipped"),
            (cv2.flip(thermal, 1), cv2.flip(rgb, 1), "都翻轉", "Both Flipped")
        ]
        
        for col_idx, (t, r, label, desc) in enumerate(combinations):
            # Resize thermal 以便比較
            t_up = cv2.resize(t, (r.shape[1], r.shape[0]), interpolation=cv2.INTER_CUBIC)
            
            # Thermal
            axes[row_idx, col_idx*2].imshow(t_up, cmap='hot')
            axes[row_idx, col_idx*2].set_title(f"{label}\nThermal", fontsize=10)
            axes[row_idx, col_idx*2].axis('off')
            
            # RGB
            axes[row_idx, col_idx*2+1].imshow(r)
            axes[row_idx, col_idx*2+1].set_title(f"{label}\nRGB", fontsize=10)
            axes[row_idx, col_idx*2+1].axis('off')
    
    cap.release()
    
    plt.suptitle('翻轉方向檢測 - 請選擇畫面對應最好的組合', fontsize=14, fontweight='bold')
    plt.tight_layout()
    
    save_path = DIAGNOSTIC_DIR / 'flip_comparison.png'
    plt.savefig(save_path, dpi=120, bbox_inches='tight')
    print(f"\n✅ 翻轉比對圖已儲存: {save_path}")
    plt.show()
    
    print(f"\n{'='*70}")
    print(f"📝 請檢查圖片，判斷哪種組合正確:")
    print(f"{'='*70}")
    print(f"  選項 1 - 原始: FLIP_THERMAL=False, FLIP_RGB=False")
    print(f"  選項 2 - T翻轉: FLIP_THERMAL=True, FLIP_RGB=False")
    print(f"  選項 3 - R翻轉: FLIP_THERMAL=False, FLIP_RGB=True")
    print(f"  選項 4 - 都翻轉: FLIP_THERMAL=True, FLIP_RGB=True")
    print(f"\n確認後請修改 Step 1 的 FLIP_THERMAL 和 FLIP_RGB 參數")
    print(f"{'='*70}")

# ========================================
# 執行翻轉檢測
# ========================================
check_flip_orientation(aligned_pairs, AVI_FILE)
